# code1: 將效果(Cosmetics資料夾)轉成binary形式

In [ ]:
#code 1: 資料轉換
#coding=utf-8
#output檔案為 Cosmetics_11.txt

import os

dirName = 'Cosmetics_09'
dirpath = 'Cosmetics/{}'.format(dirName)                           #Cosmetics/Cosmetics_11
file_array = os.listdir('{}'.format(dirpath))                      #列出Cosmetics/Cosmetics_11資料夾底下有哪些檔案(以array output)
filenum = len(file_array)                                          #以計算file_array的element個數來看dirName資料夾底下有幾個csv檔
effect_title = ["保濕","溫和低刺激","會回購","明亮. 透亮","價格實在","不引起過敏","好吸收","清爽","不黏膩","服貼","滋潤","不油膩","柔嫩","不致痘","光滑","水亮","美白","舒緩","香氣怡人","無負擔","彈力","不致粉刺","修護","不緊繃","鎮定","用量省","好上手","清新","易沖淨","清涼感"]
season_title = ["春","夏","秋","冬"]

rw=open(dirName+'.txt','a+')                                        #整理後資料存到dirName.txt
e=open('ErrorFile/'+dirName+'IOError.txt','a+')


count1 = 0
count2 = 0
for i in range(0,filenum):
    filename = file_array[i]                                        #取得dirName底下第i個csv檔的檔名
    productNo = filename.split('_')[-1]    
    csvpath = dirpath +'/'+ filename + '/' + filename + '.csv'         #取得第i個csv檔的路徑
    try:
        f = open(csvpath,'r')       
        
        for line in f.readlines():
            fileContent = line.split('|')                                    
            productID = fileContent[0]                                       # 商品 ID        
            commentID = fileContent[1]                                       # 心得 ID
            age = fileContent[2].split(' ')[1].strip()                       # 年齡
            effect_str = fileContent[3].split('：')[1].strip()               # 效果
            effect = effect_str.split('、')
            score = (fileContent[4].split('：')[1].strip('分').strip())      # 評分
            if score == '淺層體驗':
                score ='na'
            try:
                skinType = fileContent[5].split('：')[1].strip()             # 膚質
            except IndexError:
                skinType = 'na'                                              # 如果膚質那欄空白就補na
            if skinType =='':
                skinType = 'na'
            season_str = fileContent[7].split('：')[1].strip()               # 使用季節
            season = fileContent[7].split('：')[1].strip().split('、')

            effect_arr = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
            season_arr = [0,0,0,0]

            for i in range (0,len(effect)):
                if effect[i] in effect_title:
                    e_index = effect_title.index(effect[i])
                    effect_arr[e_index]=1
                else:
                    effect_arr[e_index]=0

            for j in range (0,len(season)):
                if season[0] is not '':             
                    if season[j] in season_title:
                        index = season_title.index(season[j])
                        season_arr[index]=1
                    else:
                        season_arr[index]=0
            
            rw.write(commentID + ',')
            rw.write(productID + ',')           
            rw.write(age + ',')

            for index1 in range(0,len(effect_arr)):
                rw.write(str(effect_arr[index1])+',')
            rw.write(score + ',')
            rw.write(skinType + ',')

            for index2 in range(0,len(season_arr)):
                rw.write(str(season_arr[index2])+ ',')
            rw.write('\n')
 
    except IOError:
        print 'IOError', csvpath#productID,commentID
        e.write(filename + '/' + filename + '.csv'+'\n')       #儲存讀檔失敗的檔案路徑
        count1 = count1+1
    except IndexError:                                         #跳過資料沒抓完全的
        print 'IndexError',productID,commentID
        count2 = count2 + 1      
rw.close()
f.close()
e.close()
print '----------'
print count1,count2
print 'end'

# 因一篇評論中的效果和心得是分開存放的,code2和code3是用Python以心得id當做primary key來進行檔案join(也可以直接把檔案匯入MySQL後下SQL語法join)

In [ ]:
#code2: 心得檔案(Comment資料夾)檔名轉換 1_1_37508_800215.csv 改成37508_800215.txt 的格式(因想用檔名裡的心得id來join)

import os
import re

dirName = 'Comment_09'
dirpath = 'Comment/{}'.format(dirName)               #原始心得存放資料夾
writeDir = 'transform_09'                            #改檔名後存放資料夾
file_array = os.listdir('{}'.format(dirpath))                      
filenum = len(file_array)


for i in range(0,filenum):  
    filename = file_array[i]
    productNo = filename.split('_')[2]
    reviewsNo = filename.split('_')[3].split('.')[0]
    
    with open(dirpath+'/'+ filename,'r') as f:      #原始心得檔路徑
        w = open('transform/{}/'.format(writeDir) + productNo +'_'+ reviewsNo + '.txt','a+')   #改名檔路徑
        line =  "".join(f.read().split('\r'))        #讀取原始心得檔的心得
        content = re.sub(r'\|',',',line)
        content = content.split(',', 3)
        w.write(content[0]+',')                      #寫入產品id到改名txt檔
        w.write(content[1]+',')                      #寫入心得id到改名txt檔
        w.write(content[2]+',')                      #寫入產品名到改名txt檔
        str_new = re.sub(u'[!-/]|[:-@]|[[-`]|[{-~]|！|，|\.|【|】|★','',content[3])   
        w.write(str_new + '\n')                      #寫入心得到改名txt檔
        w.close()

print 'transform end'

In [ ]:
#code3:用python實做檔案join (join效果file和心得file) 
#output 檔案路徑為 join/joint_11.txt

import os
efftxtname = 'Cosmetics_09.txt'                  #效果轉成0101後的檔案
productType = '09'
rw=open('join/join_{}.txt'.format(productType),'a+')     #join後寫入的檔案
f = open(efftxtname,'r')                   

count = 0
count2 = 0

for line1 in f.readlines():                      #從效果檔讀資料
    line1 =  "".join(line1.split('\r'))
    line1 = line1.strip()                        #取得效果檔整行資料
    productID = line1.split(',')[1]              #取得productID
    reviewsID = line1.split(',')[0]              #取得reviewsID
    #print '讀取',productID,reviewsID

    try:
        t = open('transform/transform_{}/'.format(productType)+ productID + '_' + reviewsID + '.txt','r')   #開啟productID _reviewsID.txt
        for line2 in t.readlines():
            reviews = line2.split(',')[3]
            rw.write(line1 + reviews)    #將效果檔裡的每列資料和心得檔裡的產品名及心得一起print出
            #print line1,prodName,',',reviews
        
    except IOError:
        count = count+1
        print 'IOError',productID,reviewsID
    except IndexError:
        count2 = count2+1
        print 'IndexErro',productID,reviewsID
    
rw.close()
t.close()
f.close()
print count,count2
print 'end'    



# code4: 心得斷出所需的關鍵字後將使用者勾選的效果與心得斷出來的效果做合併
#狀況1:使用者有勾選會回購, 心得中也有提到會回購 -->此筆為會回購
#狀況2:使用者沒有勾選會回購, 但心得中有提到會回購 -->此筆為會回購
#狀況3:使用者有勾選會回購, 但心得中沒有提到會回購 -->此筆為會回購

In [ ]:
#code4: 效果,膚質,個人評分等等資料與心得斷詞後得到的新效果(ex.負面效果) 合併
# output檔案為 11_effcombine.txt
import re
import jieba
import jieba.analyse
jieba.analyse.set_stop_words("merge_total.txt")   #切除無用的文字
jieba.load_userdict('dict.txt.big.txt')              #情緒文字

jieba.suggest_freq("吸收優", True)
jieba.suggest_freq("吸收劣", True)
jieba.suggest_freq("未引起過敏", True)
jieba.suggest_freq("會回購", True)
jieba.suggest_freq("用量省", True)
jieba.suggest_freq("用量大", True)
jieba.suggest_freq("溫和", True)
jieba.suggest_freq("刺激性強", True)
jieba.suggest_freq("控油優", True)
jieba.suggest_freq("控油劣", True)
jieba.suggest_freq("無負擔", True)
jieba.suggest_freq("價格合理", True)
jieba.suggest_freq("非泛白", True)   
jieba.suggest_freq("非緊繃", True)
jieba.suggest_freq("緊繃", True)
jieba.suggest_freq("洗淨優", True)
jieba.suggest_freq("洗淨劣", True)
jieba.suggest_freq("清爽", True)
jieba.suggest_freq("氣味適中", True)
jieba.suggest_freq("氣味劣", True)
jieba.suggest_freq("保濕力優", True)
jieba.suggest_freq("保濕力劣", True)    
jieba.suggest_freq("引起過敏", True)
jieba.suggest_freq("非油膩", True)
jieba.suggest_freq("油膩", True)
jieba.suggest_freq("致痘", True)
jieba.suggest_freq("非致痘", True)
jieba.suggest_freq("粉刺減少", True)
jieba.suggest_freq("黏膩", True)
jieba.suggest_freq("非黏膩", True)
jieba.suggest_freq("非暗沉", True)
jieba.suggest_freq("非清爽", True)
jieba.suggest_freq("淡化黑眼圈劣", True)
jieba.suggest_freq("淡化細紋劣", True)
jieba.suggest_freq("細紋淡化", True)
jieba.suggest_freq("價格劣", True)
jieba.suggest_freq("防曬差", True)
jieba.suggest_freq("防曬優", True)
jieba.suggest_freq("易推勻", True)
jieba.suggest_freq("難推勻", True)
jieba.suggest_freq("美白優", True)
jieba.suggest_freq("美白劣", True)
jieba.suggest_freq("不回購", True)
jieba.suggest_freq("無效果", True)
jieba.suggest_freq("舒緩", True)

productType = '09'
#r=open('test.txt','r')
r=open('join/join_'+'{}'.format(productType)+'.txt','r')
w=open('final_effect/{}_effcombine.txt'.format(productType),'a+')

for line in r.readlines():
    line = line.split(',',39)
    reviews = line[39]
   
    #print '---修改前---:', reviews
    str1= re.sub(r'[0-9a-zA-z]','',reviews)
    str1 = re.sub(r'我.{,10}(肌膚|臉).{,5}(會|是).{,20}過敏|容易.{,20}過敏|過敏.{,10}肌|(本來|本身).{,16}出油' ,'討論個人狀況',reviews)
    str1 = re.sub(r'(沒有|不會|沒).{,10}負擔|零負擔|負擔.{,10}小' ,'無負擔',str1)
    str1 = re.sub(r'(不會|沒|不|無).{,10}泛白|泛白.{,12}低' ,'非泛白',str1)
        
    #區分美白正負評
    str1 = re.sub(r'(美白|亮白|美白效果).{,20}(沒有|沒).{,22}(覺|功效|感受|明顯|差異|效果|發現|感覺|很大)|變白.{,10}一點感覺也沒有','美白劣',str1)
    str1 = re.sub(r'(沒有|沒)(.{,22})(美白|變白|美白效果)|(美白|變白|亮白|美白效果)(.{,16})(還好|普通|有限|無感|有點難)','美白劣',str1)
    str1 = re.sub(r'(美白|白)(.{,22})不(.{,10})(明顯|看得出來|好|大|感覺)|(不會|看不出)(.{,10})(美白|變白)','美白劣',str1)
    str1 = re.sub(r'(明顯|感覺|臉|膚色|皮膚|發現|肌膚)(.{,20})(變白|變亮|好白|比較白|美白|明亮|亮白|透亮|透白|白皙)|美白(.{,10})(有一點|明顯|OK|ok)|(有|有點)(.{,10})(變白|美白|透亮)','美白優',str1)
    str1 = re.sub(r'美白.{,8}無效','美白劣',str1)
    #區分柔嫩正負評
    str1 = re.sub(r'(變|很|有|摸起來|臉|肌膚|感覺|皮膚|膚完之後|敷完|特別|比較|非常).{,10}(水嫩|柔嫩|水嫩嫩|嫩|滑嫩|粉嫩|白嫩|很嫩)|(水嫩|嫩嫩)的|水嫩嫩|好嫩|水嫩.{,6}不錯','柔嫩',str1)
    #區分清粉刺正負評
    str1 = re.sub(r'(粉刺|黑頭|黑頭粉刺).{,15}(沒有|不會|沒|無).{,12}(浮上來|浮出來|浮起來|冒出|不見|拔到|拔起|改善|改進|少|好|明顯|消失|顯見|感覺)','無法清粉刺',str1)
    str1 = re.sub(r'(粉刺|黑頭|黑頭粉刺).{,16}(清不掉|有限|還在)|無法清粉刺','無法清粉刺',str1)
    str1 = re.sub(r'(不|沒).{,10}(粉刺|黑頭|黑頭粉刺).{,10}(吸出|拔出|清出|浮出)','無法清粉刺',str1)
    str1 = re.sub(r'(粉刺|黑頭|黑頭粉刺).{,10}(都|會|有|就|好像|變|明顯|真的|紛紛|後來|推|很).{,10}(減少|出來|乾淨|清除|冒出|浮出|少了|跑出|浮上|少|清光|不見|消失|好擠|溶出|浮現)','粉刺減少',str1)
    str1 = re.sub(r'可以.{,10}(粉刺|黑頭|黑頭粉刺).{,6}(浮出|拔出)|拔出.{,20}粉刺|(粉刺|黑頭|黑頭粉刺).{,16}(少了|不見)|(拔出|改善).{,24}(粉刺|黑頭|黑頭粉刺)','粉刺減少',str1)
        
    #區分溫和不刺激
    str1 = re.sub(r'(不|不覺得|沒有|低|不會).{,12}(刺激|刺痛|痛痛的|刺激性|刺激|刺刺的)|刺激.{,10}不大','溫和',str1)
    str1 = re.sub(r'(有|嚴重|感到|非常|臉|太|超|覺得|會|真的).{,10}(刺痛|刺激|痛|刺癢|刺刺)|刺激.{,10}大','刺激性強',str1)
        
    #區分控油與否
    str1 = re.sub(r'(隔天|完全).{,10}(不會|沒有|不|沒).{,10}(出油|泛油)|沒.{,10}(出油|出油)|出很少油','控油優',str1)
    str1 = re.sub(r'隔天.{,10}(依然|依舊|仍然|很).{,10}(出油|泛油)|(沒|不).{,10}(控油|抑油)','控油劣',str1)
    str1 = re.sub(r'(油份|出油).{,10}(沒有|不能|無法).{,10}(減少|變少|變緩|改善|緩和)|(依然|依舊|還是).{,10}(油光|泛油|出油)|(控油|抑油).{,10}(還好|沒|不明顯|不強)','控油劣',str1)
    str1 = re.sub(r'(可以|能|少|解決|不會|不|非常).{,10}(控油|出油)|(油份|出油).{,12}(少|改善|緩和)|(有|蠻|能).{,10}(控油|抑油)|(控油|抑油).{,12}明顯|(控油|抑油).{,12}(不錯|強|棒)|很(控油|抑油)','控油優',str1)
    str1 = re.sub(r'(隔天|過.{,10}(時間|陣子)).{,10}(油份|出油|泛油)','控油劣',str1)
        
    #油膩感
    str1 = re.sub(r'卸妝油','產品',str1)       #不像卸妝油難卸 會變非油膩難卸
    str1 = re.sub(r'(油膩感|很油)' ,'油膩',str1)
    str1 = re.sub(r'油膩膩','油膩',str1) 
    str1 = re.sub(r'不油不膩|不油膩|(不|沒有|不會).{,10}(油膩|油膩感|油)' ,'非油膩',str1)
    str1 = re.sub(r'(還|感覺)(滿|很).{,10}(油|油膩感|油膩)' ,'油膩',str1) 
        
    #區分保濕
    str1 = re.sub(r'(保濕|保濕效果).{,12}(不|沒有).{,10}(夠|好|強)|(沒有|不|無|很不).{,10}保濕' ,'保濕力劣',str1)
    str1 = re.sub(r'(保濕|保濕效果).{,16}(還好|普普|普通|還好|低|不行)|(保濕|保濕效果).{,11}沒.{,10}(感覺|效果|功效|成效)', '保濕力劣',str1)
    str1 = re.sub(r'(保濕|保濕效果).{,10}(不錯|還ok|強|好|可以|驚人)','保濕力優',str1)
    str1 = re.sub(r'(有|蠻|好|挺|相當|很|比|超|滿|非常|頗).{,8}(保濕|保濕效果)','保濕力優',str1)
        
    #區分過敏
    str1 = re.sub(r'(沒|不會|抗|無|部會|沒讓|不讓).{0,16}過敏|舒緩.{0,16}過敏|(過敏).{,8}沒|不.{,6}擔心.{,10}過敏|(不|不會)引起過敏' ,'未引起過敏',str1) 
    str1 = re.sub(r'(使用|用了).{0,10}後.{0,10}(沒|不會|無|部會|沒讓|不讓).{0,10}過敏|用.{0,10}(沒|不|無).{0,10}過敏' ,'未引起過敏',str1)
    str1 = re.sub(r'(使用|用了).{0,10}後.{0,12}過敏|用了.{0,10}過敏' ,'引起過敏',str1)
    str1 = re.sub(r'(有|導致|居然|會|嚴重|輕微|呈現|讓我).{,10}過敏' ,'引起過敏',str1)
      
    #長粉刺/痘痘
    str1 = re.sub(r'用.{,6}(後|完).{,16}(不會|沒有|無)(長|冒|爆|漲|致).{,10}(粉刺|痘|豆|閉鎖性粉刺)|用了.{,16}(也沒有|也沒|不|沒有|無).{,10}(長|冒|爆|漲|致).{,10}(粉刺|痘|豆|閉鎖性粉刺)','非致痘',str1)
    str1 = re.sub(r'自從用.{,20}(不會|沒有)(長|冒|爆|漲|導致).{,10}(粉刺|痘|豆|閉鎖性粉刺)|(不|不會|沒有).{,6}致.{,10}(粉刺|痘|豆)|用了.{,16}(不|沒有|無)(粉刺|痘|豆|閉鎖性粉刺)增加','非致痘',str1)
    str1 = re.sub(r'用.{,22}(後|完|幾天|一陣|隔天).{,16}(長|冒|爆|漲|長了).{,16}(粉刺|痘|豆|閉鎖性粉刺)|致粉刺|用.{,6}後.{,24}開始.{,10}(長|冒|爆|漲|致).{,16}(粉刺|痘|豆|閉鎖性粉刺)','致痘',str1)
    str1 = re.sub(r'(粉刺|痘痘|痘子|豆|閉鎖性粉刺).{,6}(不會|沒有)(變|更)(多|嚴重)|(不會|沒).{,10}讓.{,10}(長|冒|爆|漲|致).{,16}(粉刺|痘|豆|閉鎖性粉刺)','非致痘',str1)
    str1 = re.sub(r'自從用.{,20}(長|冒|爆|漲|致).{,16}(粉刺|痘|豆|閉鎖性粉刺)|(害|讓).{,10}(長|冒|爆|漲|致).{,10}(粉刺|痘|豆|閉鎖性粉刺)','致痘',str1)
     
    #緊繃感
    str1 = re.sub(r'(不|沒有|不會).{,16}(緊繃|繃繃|澀澀|乾澀|崩)|洗完.{,10}(不|沒有).{,12}(緊繃|繃繃|澀澀|乾澀|崩|乾)|緊繃.{,10}還好','非緊繃',str1)
    str1 = re.sub(r'(感覺|好像|覺得|有|非常|十分|實在|稍|會).{,10}非緊繃','非緊繃',str1)
    str1= re.sub(r'(感覺|好像|覺得|有|非常|十分|實在|稍|會).{,10}(繃繃|繃|澀澀|乾澀|崩)|洗完會.{,10}乾','緊繃',str1)
         
    #清潔力
    str1= re.sub(r'有沒有.{,4}乾淨','未知',str1)
    str1= re.sub(r'不.{,4}擔心.{,12}乾淨|清潔.{,10}不差|卸妝(力|效果)強|(不|沒有).{,10}殘妝','洗淨優',str1)
    str1= re.sub(r'(不會|沒有).{,10}(沖|洗|卸).{,10}不乾淨|(不會|沒有).{,10}不易(洗|沖)','洗淨優',str1)
    str1= re.sub(r'(洗|沖).{,12}(不|才|沒|才能).{,10}(乾淨|沖乾淨|洗乾淨|卸乾淨)|(沒|難|不).{,14}乾淨|清潔.{,10}差','洗淨劣',str1)                  
    str1= re.sub(r'(有|很|超|算|可|能|易|可以|好).{,10}(沖|洗).{,10}乾淨','洗淨優',str1)
    str1= re.sub(r'(蠻|很|感覺|好像|覺得|有種|非常|十分|實在|可以|臉|洗).{,10}(乾淨|乾乾淨淨|潔淨)|好沖洗','洗淨優',str1)
    str1 = re.sub(r'(卸完|洗完).{,12}非油膩|難卸的.{,14}都','洗淨優',str1) 
    str1 = re.sub(r'(卸完|卸妝|洗完|卸妝力).{,10}(不|沒有).{,10}(弱|不好)|不.{,10}難卸|難卸.{,14}(都|也|能).{,10}卸|難卸.{,16}(不殘留|不會殘留)','洗淨優',str1)
    str1 = re.sub(r'(卸完|卸妝.{,4}後|洗完|卸妝力).{,16}(弱|不強|還好|普普|非常弱)|難卸|不.{,10}好卸','洗淨劣',str1)
    str1 = re.sub(r'卸.{,10}不.{,10}(乾淨|掉)|卸不了|(卸除|卸妝).{,10}(不好|沒有很好)|有殘妝','洗淨劣',str1)       
    str1 = re.sub(r'(卸妝|清潔|洗淨|卸淨|卸除|卸|潔淨|清潔效果).{,14}(乾乾淨淨|一乾二淨|夠|很好|不錯|好|很好|不賴|棒|稍微好|強|適中|上等|佳|好|快|中上|ok|提升|優|優秀|尚優|強大|不差|略勝一籌|相當夠|上等|可以)','洗淨優',str1) 
    str1= re.sub(r'(輕鬆|輕輕|輕輕鬆鬆|輕易|輕拭|輕而易舉|容易|非常|很好).{,10}(卸除|卸掉|卸完|卸下|抹去|卸妝)|好卸','洗淨優',str1)       
        
    #氣味
    str1 = re.sub(r'(不會|沒有|不).{,10}(刺鼻|難聞|臭|不好聞|嗆鼻)|聞起來.{,10}(不錯|不差|不濃)|(沒有|沒).{,10}味道','氣味適中',str1)
    str1 = re.sub(r'(香|氣味|味道).{,10}(不|沒有).{,10}(濃|刺鼻|重|臭|令人討厭|嗆鼻)','氣味適中',str1)
    str1 = re.sub(r'(香|氣味|味道).{,10}(濃|刺鼻|重|不好|臭|嗆鼻)|會臭|嗆鼻|不.{,10}喜歡.{,10}(香|氣味|味道)|(香|氣味|味道).{,10}(不|沒有).{,6}(喜歡|好聞)','氣味劣',str1)
    str1 = re.sub(r'(香|氣味|味道).{,12}(不錯|好聞|清香)','氣味適中',str1)
    
    #用量
    str1 = re.sub(r'用量.{,10}不用.{,4}(多|大)','用量省',str1)
    str1 = re.sub(r'用量.{,10}(不|沒有).{,10}(省|多)','用量大',str1)
    str1 = re.sub(r'用量.{,12}省|省.{,4}用量','用量省',str1)      
       
    #黏膩      
    str1 = re.sub(r'(不會|沒有|無|未有).{,18}(黏|黏膩|黏黏|黏膩感|粘膩)|不黏|不.{,10}(黏膩|黏膩感|粘膩)','非黏膩',str1)
    str1 = re.sub(r'(黏黏|黏|黏膩感)','黏膩',str1)
    str1 = re.sub(r'(黏膩膩)','黏膩',str1)
    
    #吸收度
    str1 = re.sub(r'吸收.{,10}不錯','吸收優',str1)
    str1 = re.sub(r'(不|難|無法|慢).{,10}吸收|吸收.{,10}(不好|慢|不佳)','吸收劣',str1)
    str1 = re.sub(r'(好|易|馬上|快|就).{,10}吸收|吸收.{,12}(好|快)','吸收優',str1)
        
    #暗沉
    str1 = re.sub(r'(不|減少).{,10}暗沉','非暗沉',str1)
        
    #清爽
    str1 = re.sub(r'(不|沒有).{,10}清爽','非清爽',str1)
    str1 = re.sub(r'(不|沒有).{,10}悶','清爽',str1)
    str1 = re.sub(r'(感覺|有點|有).{,10}悶','非清爽',str1)
    str1 = re.sub(r'清爽度不.{,10}差','清爽',str1)
    str1 = re.sub(r'清爽度.{,10}差','非清爽',str1)
        
    #細紋
    str1 = re.sub(r'細紋.{0,20}(沒作用|沒看見|不用想|失望|沒有減少|沒有用|沒變好|看不太出|沒有消失|沒有太大改善|冤枉錢|還在|一樣存在|感覺不出|沒改善|跑出來|沒什麼用|\
                              没什麼感覺|沒有明顯|沒有淡化|沒有減淡|感覺不到|沒什麼改善)','淡化細紋劣',str1)              
    str1 = re.sub(r'細紋.{0,20}(消失|改善|不明顯|不錯|平復不少|少|淡|超棒|有功效|不見|消很多|不見)','細紋淡化',str1)        
    
    #黑眼圈
    str1 = re.sub(r'黑眼圈.{0,24}(沒有改善|沒什麼|看不出|不明顯|沒有|沒變|沒感受|沒用|都沒有|沒消|無感|減少|無法|没有效果|一樣深|擦心酸|還在|無效果|沒解決|沒有感覺|\
                    還是在|眼圈改善度似乎感覺不出來|幫助不大|有點勉強了|沒有用|還好|不顯著|還是深)','淡化黑眼圈劣',str1)
    #防曬
    str1 = re.sub(r'(沒|不會).{0,10}(曬黑|變黑)','防曬優',str1)
    str1 = re.sub(r'防曬.{0,20}(不佳|無效|沒用|不夠)|(曬黑|變黑)|防曬.{0,10}不.{0,10}好','防曬差',str1) 
    str1 = re.sub(r'防曬.{0,20}(不錯|夠力|棒|沒得說|沒話說|好)','防曬優',str1) 
        
    #推勻
    str1 = re.sub(r'推薦','recommend',str1)   #好用推薦會變易推勻
    str1 = re.sub(r'(不|沒有).{0,10}難.{0,10}(推|推開|推勻)','易推勻',str1) 
    str1 = re.sub(r'不.{0,10}(好|易).{0,10}(推|推開|推勻)|難(推|推開|推勻)|(塗|抹|推)不.{0,10}(開|勻)','難推勻',str1) 
    str1 = re.sub(r'(好|輕鬆|易|很快|馬上).{0,4}(推|推開|推勻|塗)','易推勻',str1) 
        
    #區分價格正負評
    str1 = re.sub(r'(可以|如果|要是|只是|但是|若).{,10}(便宜|平價|可愛|親民|平實|實惠|優惠).{,10}(就|更|會|些|點)|不然.{,4}(真的貴|挺貴|太貴)','價格劣',str1)
    str1 = re.sub(r'(可以|在|再).{,10}(低|便宜|平價|可愛|親民|平實|優惠)(點|一點|一些|些)','價格劣',str1)
    str1 = re.sub(r'(不過|只是|可是|但是).{,10}(價格|價錢|價位).{,10}不會(不可愛|不便宜|太貴)','價格合理',str1)
    str1 = re.sub(r'(不過|只是|可是|但是).{,10}(價格|價錢|價位).{,20}(不可愛|不便宜|太貴)|(不過|只是).{,10}貴了','價格劣',str1)
    str1 = re.sub(r'不(僅|但)便宜|不過(很|蠻)便宜|不過價錢(很|還蠻|蠻)便宜','價格合理',str1)
    str1 = re.sub(r'(不過|只是|但是).{,22}(太貴|不便宜|滿貴|蠻貴|小貴|很貴)|(價格|單價|價位|價錢)(實在|實在是)(太貴|有點貴|有點高)','價格劣',str1)
    str1 = re.sub(r'(價格|價錢|價位|單價).{,12}(不|沒)(.{,20})(ok|可愛|親民|便宜|佛心|合理|接受|美麗|實惠|平價|親人|優|實在|親切)|(沒有|不).{,12}(便宜|實惠|平價)|如果.{,12}便宜一點','價格劣',str1)
    str1 = re.sub(r'(價格|價錢|價位|單價).{,12}(不|沒)(.{,16})(貴|高|錯|卻步|負擔)|不.{,12}貴|不傷荷包|平價','價格合理',str1)
    str1 = re.sub(r'(價格|價錢|價位|單價).{,12}(高|貴)|(太|稍|有點|小|真的|很|較|好)貴|貴了點|(價格|價錢|價位|單價)(.{,12})負擔|(價格|價錢|價位|單價)能再便宜|比(.*?)貴一點','價格劣',str1)
    str1 = re.sub(r'(價格|價錢|價位|單價).{,18}(ok|可愛|親民|親切|便宜|迷人|佛心|合理|實惠|可接受|平實|平易近人|優|實在)|(非常|超|超級)便宜|經濟又實惠|物美價廉|便宜','價格合理',str1)
        
    #舒緩
    str1 = re.sub(r'鎮定','舒緩',str1)
        
    #不回購
    str1 = re.sub(r'(不會|不值得|部會|不考慮).{,10}(買|回購)|回購.{,10}(低|不高|不大)|不想回購|(不|沒).{,12}(再買|繼續買|回購)|不會再.{,10}買','不回購',str1) 
    str1 = re.sub(r'(會|值得).{,10}(再買|回購)|回購.{,12}(高|大)|想回購|會繼續買','會回購',str1)
        
    #無效果
    str1 = re.sub(r'(沒|看不出|無|看不到|未).{,14}(效果|功效|成效|效用|作用)|一點用.{,6}(沒|看不出|無|看不到)|(效果|功效|成效|效用|作用).{,10}(沒|看不出)|無效|沒效','無效果',str1) 
    str1 = re.sub(r'(沒|看不出|看不到|未).{,12}(明顯|顯著).{,12}(效果|功效|成效|效用|作用|感覺)|(沒|沒有|沒有什麼).{,6}(效果|感覺)','無效果',str1)
    str1 = re.sub(r'(效果|功效|成效|效用).{,10}不.{,6}(明顯|好|出來|顯著)|(感覺|感受)不.{,12}(效果|功效|成效|效用|作用|感覺)','無效果',str1)
    
    #print '---修改後---:',str1
    
    neweff=['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0']
    #print reviews
    seg_list = jieba.analyse.extract_tags(str1,20,withWeight = False, allowPOS = ())
    
    def oldeffect(x):            #oldeffect(x): 給予效果名稱就回傳該名稱在讀入資料的位置 
        dic1 = {u'保濕力優':3,    #利用字典紀錄該效果在效果list的位置  ex保濕在line[3]的位置(line為讀取進來的資料)
               u'溫和':4, 
               u'會回購':5, 
               u'價格合理':7,
               u'未引起過敏':8,
               u'吸收優':9,
               u'清爽':10,
               u'非黏膩':11,
               u'非油膩':14,
               u'非致痘':16,
               u'美白優':19,
               u'舒緩':20,
               u'氣味適中':21,
               u'無負擔':22,
               u'非緊繃':26,
               u'用量省':28,
               u'洗淨優':31,
               }
        return dic1.get(x)
    
    def neweffect(x):           #neweffect(x): 給予效果名稱就回傳該名稱在存放新效果list的位置
        dic2 = {
               u'保濕力劣':0,    #記錄新效果在neweff list的位致ex.保濕劣在neweff[0]的位置  (neweff為創建來存放新效果的list, 下面會用到)
               u'刺激性強':1,
               u'價格劣':2,
               u'引起過敏':3,
               u'吸收劣':4,
               u'非清爽':5,
               u'黏膩':6,
               u'油膩':7,
               u'致痘':8,
               u'美白劣':9,
               u'氣味劣':10,
               u'緊繃':11,
               u'用量大':12,
               u'洗淨劣':13,
               u'控油優':14,
               u'控油劣':15,
               u'非泛白':16,
               u'粉刺減少':17,
               u'暗沉':18,
               u'非暗沉':19,
               u'淡化黑眼圈劣':20,
               u'細紋淡化':21,
               u'淡化細紋劣':22,
               u'防曬優':23,
               u'防曬差':24,
               u'易推勻':25,
               u'難推勻':26,
               u'不回購':27,
               u'無效果':28
               }
        return dic2.get(x)
    
    for i in range (len(seg_list)):
        if oldeffect(seg_list[i]) is not None:        #用斷詞結果檢查原始效果欄, 如果斷詞結果有出現原本沒有的效果, 則將原本的效果改為1
            line[oldeffect(seg_list[i])]='1'            #ex原本沒選美白,斷詞有美白,則改選美白
        if neweffect(seg_list[i]) is not None:        #如斷詞結果有有新效果, 加入新效果
            neweff[neweffect(seg_list[i])]='1'          #ex:斷詞出現昂貴, neweff list 昂貴的位置給予1
    del line[-1]                                      #讀取進來的資料刪掉心得 (因已斷完字, 不想output原始心得串)
    #print 'old:',line
    #print 'neg:',neweff
    for m in range(len(neweff)):
        line.append(neweff[m])                        #line list[34542,6921395,22,0,0,1,1....,油性肌膚,0,0,0,0] 和 neweff list[0,0,1,..,0,1]合併 
    #print 'new:',line                                 
    line =','.join(line)                              #將最終合併結果解list
    w.write(line+'\n')
    #print line
    #print '-------'
             
    #print '\n'
r.close()
w.close()
print 'end'

# 下方為code4用到的一些小技巧測試

In [ ]:
#testing example

a = ['美白','致痘']
b = [0,0,1,1,0]
def func(x):
    dic = {'美白':0, '保濕':1, '致痘':2, '會回購':3, '控油':4}
    return dic.get(x)
for i in range (len(a)):
    if func(a[i]) is not None:
        b[func(a[i])]=1
print b

In [ ]:
#testing example

l = []
l.append('foo')
print l
l.append('bar')
print l
l.append('baz')
print l

s = ','.join(l)
print s